In [0]:
products = [ 
          # (product_id, product_name, brand_id)  
         (1, 'iPhone', 100),
         (2, 'Galaxy', 200),
         (3, 'Redme', 300), # orphan record, no matching brand
         (4, 'Pixel', 400),
]

brands = [
    #(brand_id, brand_name)
    (100, "Apple"),
    (200, "Samsung"),
    (400, "Google"),
    (500, "Sony"), # no matching products
]

In [0]:
# spark, sparkSession, entry point for DF/SQL related activities
# DataFrame is structured, means we have schema, data types, then data also....
productDf = spark.createDataFrame(data=products, 
                                    schema=["product_id", "name", "brand_id"])

productDf.printSchema() 
productDf.show() # top 20 records

root
-- product_id: long (nullable = true)
-- name: string (nullable = true)
-- brand_id: long (nullable = true)

+----------+------+--------+
product_id| name|brand_id|
+----------+------+--------+
 1|iPhone| 100|
 2|Galaxy| 200|
 3| Redme| 300|
 4| Pixel| 400|
+----------+------+--------+

In [0]:
# data frame also immutable
# dataframe has rdd internally

print(productDf.rdd.getNumPartitions())
print(productDf.rdd.collect())

8
[Row(product_id=1, name='iPhone', brand_id=100), Row(product_id=2, name='Galaxy', brand_id=200), Row(product_id=3, name='Redme', brand_id=300), Row(product_id=4, name='Pixel', brand_id=400)]

In [0]:
# python filter
df2 = productDf.filter (productDf["brand_id"] > 100)
df2.show()

+----------+------+--------+
product_id| name|brand_id|
+----------+------+--------+
 2|Galaxy| 200|
 3| Redme| 300|
 4| Pixel| 400|
+----------+------+--------+

In [0]:
df2 = productDf.filter (  (productDf["brand_id"] > 100)  & (productDf["name"] == "Redme"  ) )
df2.show()

# where is alias for filter
df2 = productDf.where (  (productDf["brand_id"] > 100)  & (productDf["name"] == "Redme"  ) )
df2.show()

+----------+-----+--------+
product_id| name|brand_id|
+----------+-----+--------+
 3|Redme| 300|
+----------+-----+--------+

+----------+-----+--------+
product_id| name|brand_id|
+----------+-----+--------+
 3|Redme| 300|
+----------+-----+--------+

In [0]:
df2 = productDf.select("product_id", "brand_id")\
               .filter( productDf["brand_id"] >= 200)
display(df2) # databricks only

product_id,brand_id
2,200
3,300
4,400


In [0]:

df2 = productDf.select("product_id", "brand_id")\
               .withColumnRenamed("product_id", "id")
display(df2) # databricks only

id,brand_id
1,100
2,200
3,300
4,400


In [0]:
# alternative option to line continuation \ using ()
df2 = (productDf
         .select("product_id", "brand_id")
         .withColumnRenamed("product_id", "id")
      )
display(df2) # databricks only

id,brand_id
1,100
2,200
3,300
4,400


In [0]:
productDf.show()
brandDf.show()

+----------+------+--------+
product_id| name|brand_id|
+----------+------+--------+
 1|iPhone| 100|
 2|Galaxy| 200|
 3| Redme| 300|
 4| Pixel| 400|
+----------+------+--------+

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4064781883343566> in <module> 
 1 productDf . show ( ) 
 ----> 2 brandDf . show ( ) 

 NameError : name 'brandDf' is not defined

In [0]:
# spark, sparkSession, entry point for DF/SQL related activities
# DataFrame is structured, means we have schema, data types, then data also....
productDf = spark.createDataFrame(data=products, 
                                    schema=["product_id", "name", "brand_id"])
brandDf = spark.createDataFrame(data=brands, 
                                    schema=["brand_id", "brand_name"])
productDf.printSchema() 
productDf.show() # top 20 records

brandDf.printSchema()
brandDf.show()

root
-- product_id: long (nullable = true)
-- name: string (nullable = true)
-- brand_id: long (nullable = true)

+----------+------+--------+
product_id| name|brand_id|
+----------+------+--------+
 1|iPhone| 100|
 2|Galaxy| 200|
 3| Redme| 300|
 4| Pixel| 400|
+----------+------+--------+

root
-- brand_id: long (nullable = true)
-- brand_name: string (nullable = true)

+--------+----------+
brand_id|brand_name|
+--------+----------+
 100| Apple|
 200| Samsung|
 400| Google|
 500| Sony|
+--------+----------+

In [0]:
# Inner Join
# Join, joining two data frames
# left and right
# productDf is left, brandDf is right
# bring joined result when the condition met
productDf.join(brandDf, productDf['brand_id'] == brandDf['brand_id'], "inner").show()

+----------+------+--------+--------+----------+
product_id| name|brand_id|brand_id|brand_name|
+----------+------+--------+--------+----------+
 1|iPhone| 100| 100| Apple|
 2|Galaxy| 200| 200| Samsung|
 4| Pixel| 400| 400| Google|
+----------+------+--------+--------+----------+

In [0]:
# outer join, left outer and right outer, full outer join
# bring all the records from left and right
# if no matches found for right, it fills with null
# if no matches found for left, it fills with null

productDf.join(brandDf, productDf['brand_id'] == brandDf['brand_id'], "outer").show()

+----------+------+--------+--------+----------+
product_id| name|brand_id|brand_id|brand_name|
+----------+------+--------+--------+----------+
 null| null| null| 500| Sony|
 1|iPhone| 100| 100| Apple|
 2|Galaxy| 200| 200| Samsung|
 4| Pixel| 400| 400| Google|
 3| Redme| 300| null| null|
+----------+------+--------+--------+----------+

In [0]:
# Left, Left Outer Join
# picks all records from left, when no matches found for right, it fills with null
productDf.join(brandDf, productDf['brand_id'] == brandDf['brand_id'], "leftouter").show()

+----------+------+--------+--------+----------+
product_id| name|brand_id|brand_id|brand_name|
+----------+------+--------+--------+----------+
 1|iPhone| 100| 100| Apple|
 2|Galaxy| 200| 200| Samsung|
 4| Pixel| 400| 400| Google|
 3| Redme| 300| null| null|
+----------+------+--------+--------+----------+

In [0]:
# Right , Right outer
# picks all records from right side, when no matches found for left, it fills null
productDf.join(brandDf, productDf['brand_id'] == brandDf['brand_id'], "rightouter").show()

+----------+------+--------+--------+----------+
product_id| name|brand_id|brand_id|brand_name|
+----------+------+--------+--------+----------+
 null| null| null| 500| Sony|
 1|iPhone| 100| 100| Apple|
 2|Galaxy| 200| 200| Samsung|
 4| Pixel| 400| 400| Google|
+----------+------+--------+--------+----------+

In [0]:
# Left Simi join
# apply the join conditions on left and right, however picks only left dataframe
# similar to inner join where we filter out right table data
productDf.join(brandDf, productDf['brand_id'] == brandDf['brand_id'], "leftsemi").show()

+----------+------+--------+
product_id| name|brand_id|
+----------+------+--------+
 1|iPhone| 100|
 2|Galaxy| 200|
 4| Pixel| 400|
+----------+------+--------+

In [0]:
# left anti join
# opposite of left semi join
# picks from left side when no matches found with right, get orphan records from left side
productDf.join(brandDf, productDf['brand_id'] == brandDf['brand_id'], "leftanti").show()

+----------+-----+--------+
product_id| name|brand_id|
+----------+-----+--------+
 3|Redme| 300|
+----------+-----+--------+

In [0]:
# cross join
# left df with n records
# right df with m records
# cartesan based conditions without predicate  n x m
# similar to inner join due the == used
productDf.join(brandDf, productDf['brand_id'] == brandDf['brand_id'], "cross").show()

print("Cross join with multiple matches")
productDf.join(brandDf, productDf['brand_id'] < brandDf['brand_id'], "cross").show()


+----------+------+--------+--------+----------+
product_id| name|brand_id|brand_id|brand_name|
+----------+------+--------+--------+----------+
 1|iPhone| 100| 100| Apple|
 2|Galaxy| 200| 200| Samsung|
 4| Pixel| 400| 400| Google|
+----------+------+--------+--------+----------+

Cross join with multiple matches
+----------+------+--------+--------+----------+
product_id| name|brand_id|brand_id|brand_name|
+----------+------+--------+--------+----------+
 1|iPhone| 100| 200| Samsung|
 1|iPhone| 100| 400| Google|
 1|iPhone| 100| 500| Sony|
 2|Galaxy| 200| 400| Google|
 2|Galaxy| 200| 500| Sony|
 3| Redme| 300| 400| Google|
 3| Redme| 300| 500| Sony|
 4| Pixel| 400| 500| Sony|
+----------+------+--------+--------+----------+

Cross join
+----------+------+--------+--------+----------+
product_id| name|brand_id|brand_id|brand_name|
+----------+------+--------+--------+----------+
 1|iPhone| 100| 100| Apple|
 1|iPhone| 100| 200| Samsung|
 1|iPhone| 100| 400| Google|
 1|iPhone| 100| 500| Sony|
 2|Galaxy| 200| 100| Apple|
 2|Galaxy| 200| 200| Samsung|
 2|Galaxy| 200| 400| Google|
 2|Galaxy| 200| 500| Sony|
 3| Redme| 300| 100| Apple|
 3| Redme| 300| 200| Samsung|
 3| Redme| 300| 400| Google|
 3| Redme| 300| 500| Sony|
 4| Pixel| 400| 100| Apple|
 4| Pixel| 400| 200| Samsung|
 4| Pixel| 400| 400| Google|
 4| Pixel| 400| 500| Sony|
+----------+------+--------+--------+----------+

In [0]:

print("Cross join")
productDf.crossJoin(brandDf).show()

Cross join
+----------+------+--------+--------+----------+
product_id| name|brand_id|brand_id|brand_name|
+----------+------+--------+--------+----------+
 1|iPhone| 100| 100| Apple|
 1|iPhone| 100| 200| Samsung|
 1|iPhone| 100| 400| Google|
 1|iPhone| 100| 500| Sony|
 2|Galaxy| 200| 100| Apple|
 2|Galaxy| 200| 200| Samsung|
 2|Galaxy| 200| 400| Google|
 2|Galaxy| 200| 500| Sony|
 3| Redme| 300| 100| Apple|
 3| Redme| 300| 200| Samsung|
 3| Redme| 300| 400| Google|
 3| Redme| 300| 500| Sony|
 4| Pixel| 400| 100| Apple|
 4| Pixel| 400| 200| Samsung|
 4| Pixel| 400| 400| Google|
 4| Pixel| 400| 500| Sony|
+----------+------+--------+--------+----------+

In [0]:
# FIXME: not working...
productDf.join( brandDf, on = "brand_id", how = "cross").show()


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o1102.join.
: java.lang.IllegalArgumentException: Unsupported join type 'rightsemi'. Supported join types include: 'inner', 'outer', 'full', 'fullouter', 'full_outer', 'leftouter', 'left', 'left_outer', 'rightouter', 'right', 'right_outer', 'leftsemi', 'left_semi', 'leftanti', 'left_anti', 'cross'.
	at org.apache.spark.sql.catalyst.plans.JoinType$.apply(joinTypes.scala:43)
	at org.apache.spark.sql.Dataset.join(Dataset.scala:963)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 IllegalArgumentException Traceback (most recent call last)
 <command-4064781883343577> in <module> 
 ----> 1 productDf . join ( brandDf , on = "brand_id" , how = "rightsemi" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in join (self, other, on, how) 
 1079 on = self . _jseq ( [ ] ) 
 1080 assert isinstance ( how , basestring ) , "how should be basestring" 
 -> 1081 jdf = self . _jdf . join ( other . _jdf , on , how ) 
 1082 return DataFrame ( jdf , self . sql_ctx ) 
 1083 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 77 raise QueryExecutionException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 78 if s . startswith ( 'java.lang.IllegalArgumentException: ' ) : 
 ---> 79 raise IllegalArgumentException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 80 raise 
 81 return deco

 IllegalArgumentException : "Unsupported join type 'rightsemi'. Supported join types include: 'inner', 'outer', 'full', 'fullouter', 'full_outer', 'leftouter', 'left', 'left_outer', 'rightouter', 'right', 'right_outer', 'leftsemi', 'left_semi', 'leftanti', 'left_anti', 'cross'."